# SingleCNN Training Script

## Import libraries

In [33]:
import sys
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Import the the following path to use user-defined modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.constants import TRAIN_ORIGINAL_FILE_PATH, TEST_ORIGINAL_FILE_PATH
from utils.utils import seed_worker
from models.single_cnn import SingleCNN

In [34]:
torch.__version__

'2.7.0+cu118'

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [36]:
g = torch.Generator()
g.manual_seed(0)

## Retrive train data

In [37]:
train_data = None
with open(TRAIN_ORIGINAL_FILE_PATH, "rb") as file:
    train_data = pickle.load(file)

In [38]:
train_data.targets = torch.tensor(train_data.targets)
train_data.cluster = train_data.targets
train_data.targets = torch.zeros_like(train_data.targets)

In [39]:
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    worker_init_fn=seed_worker,
    generator=g,
)

## Retrive test data

In [40]:
test_data = None
with open(TEST_ORIGINAL_FILE_PATH, "rb") as file:
    test_data = pickle.load(file)

In [41]:
test_data.targets = torch.tensor(test_data.targets)
test_data.cluster = test_data.targets
test_data.targets = torch.zeros_like(test_data.targets)

In [42]:
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=100,
    shuffle=True,
    num_workers=2,
    worker_init_fn=seed_worker,
    generator=g,
)

## Create the model

In [43]:
model = SingleCNN(input_channels=3, num_classes=10).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

## Criterion

In [44]:
criterion = nn.CrossEntropyLoss()

## Train the model

In [45]:
best_acc = 0  # best test accuracy
best_acc_list = []
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [46]:
def train(epoch):
    global model, optimizer, scheduler, criterion
    print(f"\nTraining - Epoch: {epoch}")

    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        print(f"Training - Epoch: {epoch}, Batch: {batch_idx}")

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # for optim in optimizers:
        #     optim.zero_grad()

        # if args.mixture:
        #     outputs, _, loss, _ = net(inputs)
        #     loss = criterion(outputs, targets) + 0.01 * loss
        # else:
        #     if args.model == "resnet18":
        #         outputs, _ = net(inputs)
        #     else:
        #         outputs = net(inputs)

        outputs = model(inputs)
        print(f"Outputs 10x10 block:\n{outputs[10:20, 10:20]}")
        print(f"Targets 10x10 block:\n{targets[10:20, 10:20]}")

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        # for optim in optimizers:
        #     optim.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

In [47]:
def test(epoch):
    global best_acc, model, optimizer, scheduler, criterion
    print(f"\nTesting - Epoch: {epoch}")

    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            print(f"Testing - Epoch: {epoch}, Batch: {batch_idx}")

            inputs, targets = (inputs.to(device), targets.to(device))
            # if args.mixture:
            #     outputs, select0, _, _ = net(inputs)
            # else:
            #     if args.model == "resnet18":
            #         outputs, _ = net(inputs)
            #     else:
            #         outputs = net(inputs)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            print(f"Correct: {correct}, Total: {total}")

    # Save checkpoint.
    acc = 100.0 * correct / total
    if acc > best_acc:
        print("Saving...")
        state = {
            "net": model.state_dict(),
            "acc": acc,
            "epoch": epoch,
        }
        if not os.path.isdir("checkpoint"):
            os.mkdir("checkpoint")
        torch.save(state, "./checkpoint/ckpt.pth")
        best_acc = acc

In [48]:
for epoch in range(0, 1):
    train(epoch)
    test(epoch)
    scheduler.step()

best_acc_list.append(best_acc)
best_acc = 0

print(
    f"Average accuracy: {np.mean(best_acc_list)} \t standard deviation: {np.std(best_acc_list)}"
)


Training - Epoch: 0
Training - Epoch: 0, Batch: 0
Outputs 10x10 block:
tensor([], device='cuda:0', size=(10, 0), grad_fn=<SliceBackward0>)


IndexError: too many indices for tensor of dimension 1